In [13]:
# Ensure project root (OpenCEP) is on sys.path for imports like `from base.Pattern import Pattern`
import sys, os, pathlib

nb_dir = (
    pathlib.Path(__file__).parent if "__file__" in globals() else pathlib.Path.cwd()
)
project_root = str((nb_dir / "..").resolve())
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [14]:
from datetime import timedelta
from CEP import CEP
from base.Pattern import Pattern
from base.PatternStructure import (
    SeqOperator,
    PrimitiveEventStructure,
    KleeneClosureOperator,
)
from condition.CompositeCondition import AndCondition
from condition.BaseRelationCondition import SmallerThanCondition
from condition.Condition import Variable, SimpleCondition
from condition.KCCondition import KCIndexCondition
from stream.FileStream import FileInputStream, FileOutputStream
from plugin.citibike.CitiBike import (
    CitiBikeTripEventTypeClassifier,
    CitiBikeDataFormatter,
)
import test

In [15]:
"""citibikeConsecutiveRidesPattern = Pattern(
    SeqOperator(
        PrimitiveEventStructure("CitiBikeTrip", "a"),
        PrimitiveEventStructure("CitiBikeTrip", "b"),
    ),
    AndCondition(
        SimpleCondition(
            Variable("a", lambda x: (x["startstationid"], x["endstationid"])),
            Variable("b", lambda x: (x["startstationid"], x["endstationid"])),
            relation_op=lambda a, b: a[1] == b[0],
        )
    ),
    timedelta(minutes=5),
)"""

'citibikeConsecutiveRidesPattern = Pattern(\n    SeqOperator(\n        PrimitiveEventStructure("CitiBikeTrip", "a"),\n        PrimitiveEventStructure("CitiBikeTrip", "b"),\n    ),\n    AndCondition(\n        SimpleCondition(\n            Variable("a", lambda x: (x["startstationid"], x["endstationid"])),\n            Variable("b", lambda x: (x["startstationid"], x["endstationid"])),\n            relation_op=lambda a, b: a[1] == b[0],\n        )\n    ),\n    timedelta(minutes=5),\n)'

In [16]:
citibikeHotPathsPattern = Pattern(
    SeqOperator(
        KleeneClosureOperator(PrimitiveEventStructure("CitiBikeTrip", "a")),
        PrimitiveEventStructure("CitiBikeTrip", "b"),
    ),
    AndCondition(
        KCIndexCondition(
            names={"a"},
            getattr_func=lambda x: x["bike_id"],
            relation_op=lambda a1, a2: a1 == a2,
            offset=-1,
        ),
        KCIndexCondition(
            names={"a"},
            getattr_func=lambda x: (
                int(float(x["startstationid"])),
                int(float(x["endstationid"])),
            ),
            relation_op=lambda a1, a2: a1[0] == a2[1],
            offset=-1,
        ),
        SimpleCondition(
            Variable("a", lambda x: x[-1]["bike_id"]),
            Variable("b", lambda x: x["bike_id"]),
            relation_op=lambda a, b: a == b,
        ),
        SimpleCondition(
            Variable("a", lambda x: int(float(x[-1]["endstationid"]))),
            Variable("b", lambda x: int(float(x["startstationid"]))),
            relation_op=lambda a, b: a == b,
        ),
        SimpleCondition(
            Variable("b", lambda x: int(float(x["endstationid"]))),
            relation_op=lambda end_id: str(end_id) in {"484"},
        ),
    ),
    timedelta(minutes=60),
)

In [17]:
from parallel.ParallelExecutionParameters import (
    DataParallelExecutionParametersHirzelAlgorithm,
)
from parallel.ParallelExecutionPlatforms import ParallelExecutionPlatforms


"""dp_params = DataParallelExecutionParametersHirzelAlgorithm(
    platform=ParallelExecutionPlatforms.THREADING,
    units_number=8,  # how many threads
    key="bike_id",
)"""
# cep = CEP([citibikeHotPathsPattern], parallel_execution_params=dp_params)
cep = CEP([citibikeHotPathsPattern])
events = FileInputStream("../test/EventFiles/201901-citibike-tripdata-test.txt")

Creating evaluation manager...
 - Parallel execution: None
 - Storage: None
 - Using ParallelExecutionModes.SEQUENTIAL execution mode
!!! Using default evaluation mechanism parameters...
!!! Creating tree-based evaluation mechanism...
Tree calling create_storage_unit with storage_params: TreeStorageParameters(sort_storage=False, attributes_priorities={}, clean_up_interval=10, prioritize_sorting_by_timestamp=True, enable_load_shedding=False, load_shedding_threshold=1000, load_shedding_drop_rate=0.1, load_shedding_strategy=random)
InternalNode creating storage: sort=False, sorting_key=None
UnsortedPatternMatchStorage created with storage_params: TreeStorageParameters(sort_storage=False, attributes_priorities={}, clean_up_interval=10, prioritize_sorting_by_timestamp=True, enable_load_shedding=False, load_shedding_threshold=1000, load_shedding_drop_rate=0.1, load_shedding_strategy=random)
InternalNode creating storage: sort=False, sorting_key=None
UnsortedPatternMatchStorage created with s

In [18]:
cep.run(
    events,
    FileOutputStream("../test/demo/Matches", "output_citibike_bon.txt"),
    CitiBikeDataFormatter(),
)

Starting CEP evaluation...
Using generic file input stream processing
Parsing raw data: 320,01:47.4,07:07.6,3160,,3283,,15839



ValueError: Unable to parse timestamp in event: 320,01:47.4,07:07.6,3160,,3283,,15839
